In [3]:
import pandas as pd
import os
import json

Спроба обчислення відстані Левенштайна для тестового прикладу і огляд результатів

In [22]:
from Levenshtein import distance

In [31]:
for file_name in os.listdir('data'):
    
    if('.csv' in file_name):
        print(file_name)
        df = pd.read_csv('data\\' + file_name, sep = ';')
        df['distance'] = df.apply(lambda x: distance(x.FULL_NAME, 'Приватне підприємство "Магістар-СГ"'), axis = 1)
        print(df.distance.describe())
        print(df.sort_values(by = 'distance')[['FULL_NAME', 'distance']])

J0900108(S0100113) Balans forma 1.csv
count    18667.000000
mean        62.266031
std         23.994326
min         27.000000
25%         48.000000
50%         55.000000
75%         71.000000
max        242.000000
Name: distance, dtype: float64
                                               FULL_NAME  distance
11179                                       ПрАТ "Літак"        27
15496                   ПРИВАТНЕ ПIДПРИЄМСТВО "МАРІАМ-С"        27
4826                      ПРИВАТНЕ ПІДПРИЄМСТВО "МІКС-2"        28
10698                     ПРИВАТНЕ ПІДПРИЄМСТВО "МВМ-13"        28
8072                  ПРИВАТНЕ ПIДПРИЄМСТВО "МАРКЕТФЕСТ"        28
...                                                  ...       ...
8986   ПЕРВИННА ПРОФСПІЛКОВА ОРГАНІЗАЦІЯ ВИРОБНИЧОГО ...       220
16320  КОМУНАЛЬНЕ НЕКОМЕРЦІЙНЕ ПІДПРИЄМСТВО "КИЇВСЬКИ...       225
15132  ПIДПРИЄМСТВО ОБ'ЄДНАННЯ ГРОМАДЯН (РЕЛIГIЙНОЇ О...       226
6487   РЕЛIГIЙНА ОРГАНIЗАЦIЯ "РЕЛІГІЙНА ГРОМАДА ЛЬВІВ...       232
6992   ВІДДІЛ ЖИТЛ

Коментар: 
1) подумав, що краще пошукати відшкальовану відстань, бо для абсолютних значень як тут потрібно додатково аналізувати оптимальне значення.
2) бачу, що є багато слів, які будуть заважати порівнювати іменна компаній(наприклад "приватне підприємство"

In [41]:
from pyjarowinkler import distance

Вирішив спробувати відстань Jaro

In [42]:
distance.get_jaro_distance('cat', 'cut')

0.8

In [44]:
for file_name in os.listdir('data'):
    
    if('.csv' in file_name):
        print(file_name)
        df = pd.read_csv('data\\' + file_name, sep = ';')
        df['distance'] = df.apply(lambda x: distance.get_jaro_distance(x.FULL_NAME, 'Приватне підприємство "Магістар-СГ"', winkler=True), axis = 1)
        print(df.distance.describe())
        print(df.sort_values(by = 'distance')[['FULL_NAME', 'distance']])

J0900108(S0100113) Balans forma 1.csv
count    18667.000000
mean         0.543825
std          0.075507
min          0.290000
25%          0.500000
50%          0.530000
75%          0.570000
max          0.920000
Name: distance, dtype: float64
                                               FULL_NAME  distance
14608                                    КОНСОРЦIУМ "БК"      0.29
3498   КОМУНАЛЬНА ОРГАНIЗАЦIЯ (УСТАНОВА, ЗАКЛАД) Кому...      0.33
4750   КОМУНАЛЬНИЙ ЗАКЛАД ЗАКЛАД ДОШКІЛЬНОЇ ОСВІТИ (Д...      0.34
2913   КОМУНАЛЬНИЙ ЗАКЛАД ПИЛИПОВИЦЬКИЙ ЗАКЛАД ДОШКІЛ...      0.34
994    КОМУНАЛЬНИЙ ЗАКЛАД "ДОШКІЛЬНИЙ НАВЧАЛЬНИЙ ЗАКЛ...      0.34
...                                                  ...       ...
9109                       ПРИВАТНЕ ПIДПРИЄМСТВО "МАТЛА"      0.91
16515                  ПРИВАТНЕ ПIДПРИЄМСТВО "МТ-СЕРВІС"      0.91
6755                       ПРИВАТНЕ ПІДПРИЄМСТВО "ЮЛМАС"      0.91
12113                        ПРИВАТНЕ ПIДПРИЄМСТВО "АСТ"      0.91
4826              

Коментар: загалом відшкальованість більше мені подобається, але фільтрацію слів потрібно буде точно зробити

Переглядаю наскільки " поширені

In [47]:
for file_name in os.listdir('data'):
    if('.csv' in file_name):
        print(file_name)
        df = pd.read_csv('data\\' + file_name, sep = ';')
        print(df.apply(lambda x: not '"' in x.FULL_NAME, axis = 1).any())

J0900108(S0100113) Balans forma 1.csv
True
J0900207(S0100213) Zvit finrez forma 2.csv
True
J0901107(S0110011) Finzvit malogo pidpr forma 1-m.csv
True
J0901205(S0111003) Finzvit mikropidpr forma 1-ms.csv
True
J0901702 konsol balans forma 1-k.csv
True
J0901801 konsol finzvit rezult forma 2-k.csv
True
J0902203 Balans konsol bank.csv
False
J0902303 Zvit konsol bank.csv
False
J0903203 Balans forma1-ds.csv
True
J0903303 Zvit finrez forma 2-ds.csv
True


In [48]:
for file_name in os.listdir('data'):
    if('.csv' in file_name):
        print(file_name)
        df = pd.read_csv('data\\' + file_name, sep = ';')
        print(df[(df.apply(lambda x: not '"' in x.FULL_NAME, axis = 1))].head(5))

J0900108(S0100113) Balans forma 1.csv
         TIN                                          FULL_NAME FORM_CODE  \
2    7686118                            ВІЙСЬКОВА ЧАСТИНА А1603  S0100113   
20  43794262  Джулинська сільська територіальна виборча комі...  J0900108   
24   4329783                         ГАВРИШІВСЬКА СІЛЬСЬКА РАДА  J0900108   
32  33074954            САДІВНИЧО-ДАЧНЕ ТОВАРИСТВО '' КРАСНЕ ''  J0900108   
43   3338900       ГАЙСИНСЬКИЙ КОМБІНАТ КОМУНАЛЬНИХ ПІДПРИЄМСТВ  S0100113   

                            FORM_NAME  PERIOD_MONTH  PERIOD_YEAR       D_GET  \
2   Баланс (Звiт про фiнансовий стан)            12         2020  01.03.2021   
20  Баланс (Звiт про фiнансовий стан)            12         2020  01.03.2021   
24  Баланс (Звiт про фiнансовий стан)            12         2020  02.03.2021   
32  Баланс (Звiт про фiнансовий стан)            12         2020  19.01.2021   
43  Баланс (Звiт про фiнансовий стан)            12         2020  12.02.2021   

    R1000G3  R1000

Оглядаю частоти вживання слів

In [62]:
words_frequency = dict()

for file_name in os.listdir('data'):
    if('.csv' in file_name):
        print(file_name)
        df = pd.read_csv('data\\' + file_name, sep = ';')
        for row in df.iterrows():
            for word in row[1].FULL_NAME.lower().split():
                if(word in words_frequency.keys()):
                    words_frequency[word] += 1
                else:
                    words_frequency[word] = 1

J0900108(S0100113) Balans forma 1.csv
J0900207(S0100213) Zvit finrez forma 2.csv
J0901107(S0110011) Finzvit malogo pidpr forma 1-m.csv
J0901205(S0111003) Finzvit mikropidpr forma 1-ms.csv
J0901702 konsol balans forma 1-k.csv
J0901801 konsol finzvit rezult forma 2-k.csv
J0902203 Balans konsol bank.csv
J0902303 Zvit konsol bank.csv
J0903203 Balans forma1-ds.csv
J0903303 Zvit finrez forma 2-ds.csv


In [65]:
sorted(words_frequency.items(), key=lambda x:x[1], reverse = True)

[('товариство', 247336),
 ('з', 238053),
 ('обмеженою', 231052),
 ('вiдповiдальнiстю', 218561),
 ('приватне', 39406),
 ('пiдприємство', 36983),
 ("об'єднання", 29253),
 ('організація', 28144),
 ('громадська', 27941),
 ('органiзацiя', 27270),
 ('будинку', 25200),
 ('багатоквартирного', 25034),
 ('області', 23864),
 ('спiввласникiв', 22864),
 ('ради', 22207),
 ('підприємство', 22201),
 ('району', 16597),
 ('рада', 14117),
 ('та', 13750),
 ('відповідальністю', 13171),
 ('кооператив', 13166),
 ('сільська', 13123),
 ('міської', 12818),
 ('україни', 11007),
 ('центр', 10623),
 ('профспілкова', 10495),
 ('управління', 10357),
 ('комунальне', 10169),
 ('заклад', 9836),
 ('державної', 9772),
 ('-', 9586),
 ('первинна', 9451),
 ('громада', 8745),
 ('компанія', 8615),
 ('церкви', 8402),
 ('фонд', 8152),
 ('районна', 7720),
 ('і', 7626),
 ('"', 7428),
 ('фірма', 7363),
 ('україна"', 7130),
 ('у', 7081),
 ('груп"', 7080),
 ('української', 6778),
 ('релігійна', 6160),
 ('районної', 6101),
 ('адмініс

Надзвичайно багато однакових слів
Далі будую фільтр

In [76]:
def filtering_func(pair):
    key, value = pair
    if value > 10000:
        return True
    else:
        return False

In [79]:
stop_words = dict(filter(filtering_func, words_frequency.items()))

In [80]:
stop_words

{'підприємство': 22201,
 "об'єднання": 29253,
 'приватне': 39406,
 'товариство': 247336,
 'з': 238053,
 'обмеженою': 231052,
 'вiдповiдальнiстю': 218561,
 'комунальне': 10169,
 'пiдприємство': 36983,
 'міської': 12818,
 'ради': 22207,
 'відповідальністю': 13171,
 'сільська': 13123,
 'району': 16597,
 'області': 23864,
 'центр': 10623,
 'та': 13750,
 'рада': 14117,
 'україни': 11007,
 'громадська': 27941,
 'органiзацiя': 27270,
 'багатоквартирного': 25034,
 'будинку': 25200,
 'організація': 28144,
 'профспілкова': 10495,
 'спiввласникiв': 22864,
 'управління': 10357,
 'кооператив': 13166}

Тестування фільтру

In [99]:
def remove_stop_words(s):
    for word in stop_words.keys():
        s = s.replace(word, '')
    s = s.strip()
    if(len(s) == 0):
        s= 's'
    return s

In [100]:
remove_stop_words('приватне пiдприємство "патар"')

'"пар"'

Тест фільтру з тим самим прикладом

In [101]:
for file_name in os.listdir('data'):
    
    if('.csv' in file_name):
        print(file_name)
        df = pd.read_csv('data\\' + file_name, sep = ';')
        df['FULL_NAME'] = df['FULL_NAME'].str.lower().apply(remove_stop_words)
        df['distance'] = df.apply(lambda x: distance.get_jaro_distance(x.FULL_NAME, remove_stop_words('Приватне підприємство "Магістар-СГ"'.lower()), winkler=True), axis = 1)
        print(df.distance.describe())
        print(df.sort_values(by = 'distance')[['FULL_NAME', 'distance']])

J0900108(S0100113) Balans forma 1.csv
count    18667.000000
mean         0.483695
std          0.108783
min          0.000000
25%          0.400000
50%          0.470000
75%          0.560000
max          0.930000
Name: distance, dtype: float64
                                               FULL_NAME  distance
2906   будинко № 9 макарівської квартирно-експлуаційн...      0.00
1708   повне  куьміна д.в.  щербак г.г. "фараон"-ломбард      0.00
11251                                      половинчицька      0.00
10512                  полiтична партiя "справедливість"      0.00
4164                  будинко № 1 івано-франківської кеч      0.00
...                                                  ...       ...
3328                                            "маргіт"      0.85
2086                                            "магреф"      0.86
3525                                 "магістраль-сервіс"      0.86
1937                                            "магрок"      0.86
9070              